In [1]:
import time
import json
import requests

from redis import Redis

In [2]:
DEFAULT_TTL = 60

redis = Redis(host="localhost", port=6379)

In [3]:
def buscar_municipios(uf, as_dict=True):
    uri_municipios = "https://servicodados.ibge.gov.br/api/v1/localidades/estados/{}/municipios"
    uri = uri_municipios.format(uf)
    
    result = requests.get(uri, headers={ "Accept": "application/json" })
    
    if as_dict:
        return result.json()
    else:
        return result.content

In [4]:
def buscar_municipios_com_cache(uf):
    key = "api_municipios:municipios:{}".format(uf)
    
    municipios = redis.get(key)
    
    if municipios == None:
        print("Cache Miss")
        municipios = buscar_municipios(uf, as_dict=False)
        redis.setex(key, DEFAULT_TTL, municipios)
    else:
        print("Cache Hit")
    
    return json.loads(municipios)

In [8]:
UF = "SP"

In [9]:
inicio = time.time()
municipios = buscar_municipios_com_cache(UF)
fim = time.time()

diferenca_ms = (fim - inicio) * 1000

print("Duração: {} ms".format(diferenca_ms))

municipios

Cache Miss
Duração: 285.8600616455078 ms


[{'id': '3500105',
  'nome': 'Adamantina',
  'microrregiao': {'id': 35035,
   'nome': 'Adamantina',
   'mesorregiao': {'id': 3508,
    'nome': 'Presidente Prudente',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}},
  'regiao-imediata': {'id': 350019,
   'nome': 'Adamantina - Lucélia',
   'regiao-intermediaria': {'id': 3505,
    'nome': 'Presidente Prudente',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}},
 {'id': '3500204',
  'nome': 'Adolfo',
  'microrregiao': {'id': 35004,
   'nome': 'São José do Rio Preto',
   'mesorregiao': {'id': 3501,
    'nome': 'São José do Rio Preto',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}},
  'regiao-imediata': {'id': 350025,
   'nome': 'São José do Rio Preto',
   'regiao-intermediaria': {'id': 3507,
    'nome'